This notebook takes postal codes from Toronto and adds coordinates from BING API to them.

In [18]:
#pip install geocoder

In [19]:
import pandas as pd
import numpy as np
import os
from geopy.geocoders import Nominatim
import geocoder

## Import Data

In [20]:
# View files in directory
os.listdir()

['.DS_Store',
 'Add_Coordinates.ipynb',
 'Icon\r',
 'Toronto_Postal_Codes.ipynb',
 'Dataset_PC_with_Coord.csv',
 'Dataset_Toronto_Postal_Codes.csv',
 'README.md',
 '.gitattributes',
 '.ipynb_checkpoints',
 '.git']

In [21]:
# Import data
postal_codes = pd.read_csv('Dataset_Toronto_Postal_Codes.csv')
postal_codes.sample(5)

,PostalCode,Borough,Neighborhood
42,M4L,East Toronto,"India Bazaar, The Beaches West"
45,M4P,Central Toronto,Davisville North
91,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
81,M6N,York,"Runnymede, The Junction North"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


## Get location coordinates

In [22]:
# Unique postal codes
unique_postal_codes = postal_codes.PostalCode.unique()
unique_postal_codes

array(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M',
       'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H',
       'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B',
       'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B',
       'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N',
       'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A',
       'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M',
       'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A',
       'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M',
       'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W',
       'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N',
       'M9P', 'M9R', 'M9V', 'M9W'], dtype=object)

#### Request from BING API

In [23]:
# Coordinate dictionary to store postal code coordinate data
coords = {}

# Get coordenates for all codes
for pc in unique_postal_codes:

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.bing('{}, Toronto, Ontario'.format(pc),key=BING_KEY)
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    coords[pc] = lat_lng_coords

#### Transform into DataFrame

In [24]:
coordinates = pd.DataFrame(coords).transpose()
coordinates.columns = ['Latitude','Longitude']
coordinates

,Latitude,Longitude
M1B,43.810154,-79.194603
M1C,43.784672,-79.158958
M1E,43.766289,-79.172890
M1G,43.768288,-79.214111
M1H,43.769180,-79.238770
...,...,...
M9N,43.705570,-79.518616
M9P,43.696484,-79.529266
M9R,43.684505,-79.557915
M9V,43.742935,-79.584831


## Add coordinates to Postal Codes DataFrame

In [25]:
postal_codes = postal_codes.merge(coordinates,
                                  how='left',
                                  left_on='PostalCode',
                                  right_index=True)

In [26]:
postal_codes

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.810154,-79.194603
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784672,-79.158958
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.766289,-79.172890
3,M1G,Scarborough,Woburn,43.768288,-79.214111
4,M1H,Scarborough,Cedarbrae,43.769180,-79.238770
...,...,...,...,...,...
98,M9N,York,Weston,43.705570,-79.518616
99,M9P,Etobicoke,Westmount,43.696484,-79.529266
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.684505,-79.557915
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.742935,-79.584831


### Export csv

In [27]:
postal_codes.to_csv('Dataset_PC_with_Coord.csv',index=False)